### Preliminary work: Importing required packages, importing parameters setting, defining functions

In [ ]:
# Required packages
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.stats import norm
import pdb
import warnings
import time
import sys
from scipy.interpolate import RegularGridInterpolator, CubicSpline
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
sys.stdout.flush()
import os
import pickle
import SolveLinSys

In [ ]:
def finiteDiff(data, dim, order, dlt, cap = None):  # compute the central difference derivatives for given input and dimensions
    """
This function computes up to second order derivatives via finite difference scheme

Input arguments are:
Data: Function value grids which you want to compute derivatives
dim: specify which dimension you want to compute derivatives in
order: denotes order of the derivative
dlt: specify the intervals of finite difference scheme
cap: denotes whether you want to cap derivatives to some minimum values

By Jiaming Wang (Jiamingwang@uchicago.edu)
    """
    res = np.zeros(data.shape)
    if order == 1:                    # first order derivatives
        
        if dim == 0:                  # to first dimension

            res[1:-1,:,:] = (1 / (2 * dlt)) * (data[2:,:,:] - data[:-2,:,:])
            res[-1,:,:] = (1 / dlt) * (data[-1,:,:] - data[-2,:,:])
            res[0,:,:] = (1 / dlt) * (data[1,:,:] - data[0,:,:])

        elif dim == 1:                # to second dimension

            res[:,1:-1,:] = (1 / (2 * dlt)) * (data[:,2:,:] - data[:,:-2,:])
            res[:,-1,:] = (1 / dlt) * (data[:,-1,:] - data[:,-2,:])
            res[:,0,:] = (1 / dlt) * (data[:,1,:] - data[:,0,:])

        elif dim == 2:                # to third dimension

            res[:,:,1:-1] = (1 / (2 * dlt)) * (data[:,:,2:] - data[:,:,:-2])
            res[:,:,-1] = (1 / dlt) * (data[:,:,-1] - data[:,:,-2])
            res[:,:,0] = (1 / dlt) * (data[:,:,1] - data[:,:,0])

        else:
            raise ValueError('wrong dim')
            
    elif order == 2:
        
        if dim == 0:                  # to first dimension

            res[1:-1,:,:] = (1 / dlt ** 2) * (data[2:,:,:] + data[:-2,:,:] - 2 * data[1:-1,:,:])
            res[-1,:,:] = (1 / dlt ** 2) * (data[-1,:,:] + data[-3,:,:] - 2 * data[-2,:,:])
            res[0,:,:] = (1 / dlt ** 2) * (data[2,:,:] + data[0,:,:] - 2 * data[1,:,:])

        elif dim == 1:                # to second dimension

            res[:,1:-1,:] = (1 / dlt ** 2) * (data[:,2:,:] + data[:,:-2,:] - 2 * data[:,1:-1,:])
            res[:,-1,:] = (1 / dlt ** 2) * (data[:,-1,:] + data[:,-3,:] - 2 * data[:,-2,:])
            res[:,0,:] = (1 / dlt ** 2) * (data[:,2,:] + data[:,0,:] - 2 * data[:,1,:])

        elif dim == 2:                # to third dimension

            res[:,:,1:-1] = (1 / dlt ** 2) * (data[:,:,2:] + data[:,:,:-2] - 2 * data[:,:,1:-1])
            res[:,:,-1] = (1 / dlt ** 2) * (data[:,:,-1] + data[:,:,-3] - 2 * data[:,:,-2])
            res[:,:,0] = (1 / dlt ** 2) * (data[:,:,2] + data[:,:,0] - 2 * data[:,:,1])

        else:
            raise ValueError('wrong dim')
        
    else:
        raise ValueError('wrong order')
        
    if cap is not None:
        res[res < cap] = cap
    return res

def quad_points_legendre(n):
    u = np.sqrt(1 / (4 - 1 / np.linspace(1,n-1,n-1)**2))  # upper diag
    [lambda0,V] = np.linalg.eig(np.diagflat(u,1) + np.diagflat(u,-1))  # V's column vectors are the main d
    i = np.argsort(lambda0)
    Vtop = V[0,:]
    Vtop = Vtop[i]
    w = 2 * Vtop ** 2
    return (lambda0[i],w)

def quad_points_hermite(n):
    i = np.linspace(1,n-1,n-1)
    a = np.sqrt(i / 2.0)
    [lambda0,V] = np.linalg.eig(np.diagflat(a,1) + np.diagflat(a,-1))
    i = np.argsort(lambda0)
    Vtop = V[0,:]
    Vtop = Vtop[i]
    w = np.sqrt(np.pi) * Vtop ** 2
    return (lambda0[i],w)


def quad_int(f,a,b,n,method):
    """
This function takes a function f to integrate from the multidimensional
interval specified by the row vectors a and b. N different points are used
in the quadrature method. Legendre and Hermite basis functions are
currently supported. In the case of Hermite methodology b is the normal
density and a is the normal mean.

Created by John Wilson (johnrwilson@uchicago.edu) & Updated by Jiaming Wang (Jiamingwang@uchicago.edu)
    """
    if method == 'legendre':
        
        (xs,ws) = quad_points_legendre(n)
        g = lambda x: f((b-a) * 0.5  * x + (a + b) * 0.5)
        s = np.prod((b-a) * 0.5)                
        
    elif method == 'hermite':
        
        (xs,ws) = quad_points_hermite(n)
        g = lambda x: f(np.sqrt(2) * b * x + a)
        s = 1 / np.sqrt(np.pi)
        
    else:
        raise TypeError('Wrong polynomials specification')
    
    
    tp = type(a)
    if tp is np.float64 or tp is int or tp is np.double:
        res = 0
        for i in range(n):
#             pdb.set_trace()
            res += ws[i] * g(xs[i])
    else:
        raise ValueError('dimension is not 1')
    
    return s * res


def cap(x, lb, ub):
    if x <= ub or x >= lb:
        return x
    else:
        if x > ub:
            return ub
        else:
            return lb

In [ ]:
def PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, v0, ε = 1, solverType = 'False Transient'):

    if solverType == 'False Transient':
        A = A.reshape(-1,1,order = 'F')
        B = np.hstack([B_r.reshape(-1,1,order = 'F'),B_f.reshape(-1,1,order = 'F'),B_k.reshape(-1,1,order = 'F')])
        C = np.hstack([C_rr.reshape(-1,1,order = 'F'), C_ff.reshape(-1,1,order = 'F'), C_kk.reshape(-1,1,order = 'F')])
        D = D.reshape(-1,1,order = 'F')
        v0 = v0.reshape(-1,1,order = 'F')
        v1 = v0.reshape(-1,1,order = 'F')
        out = SolveLinSys.solveFT(stateSpace, A, B, C, D, v0, ε)

        return out

    elif solverType == 'Feyman Kac':
        
        A = A.reshape(-1, 1, order='F')
        B = np.hstack([B_r.reshape(-1, 1, order='F'), B_f.reshape(-1, 1, order='F'), B_k.reshape(-1, 1, order='F')])
        C = np.hstack([C_rr.reshape(-1, 1, order='F'), C_ff.reshape(-1, 1, order='F'), C_kk.reshape(-1, 1, order='F')])
        D = D.reshape(-1, 1, order='F')
        v0 = v0.reshape(-1, 1, order='F')
        out = SolveLinSys.solveFK(stateSpace, A, B, C, D, v0)

        return out

### Solving HJB for Preference Damage

In [ ]:
check = loadmat('/Users/han/Dropbox/share with John/Final Code/Preference/hjb_solution/Averse_weighted.mat')
v0_guess = check['out_comp']
q_guess = check['q']
e_guess = check['e_hat']

In [ ]:
start_time = time.time()

# Damage function choics
damageSpec = 'Weighted'  # Choose among "High"(Weitzman), 'Low'(Nordhaus) and 'Weighted'
if damageSpec == 'High':
    weight = 0.0
elif damageSpec == 'Low':
    weight = 1.0
else:
    weight = 0.5

ξₚ =  1 / 4000  # Ambiguity Averse Paramter 
# Sensible choices are from 0.0002 to 4000, while for parameters input over 0.01 the final results won't alter as much
    
McD = np.loadtxt('./data/TCRE_MacDougallEtAl2017_update.txt')
par_lambda_McD = McD / 1000

β𝘧 = np.mean(par_lambda_McD)  # Climate sensitivity parameter, MacDougall (2017)
σᵦ = np.var(par_lambda_McD, ddof = 1)  # varaiance of climate sensitivity parameters
λ = 1.0 / σᵦ 
δ = 0.01        # subjective rate of discount
κ = 0.032       # 
σ𝘨 = 0.02
σ𝘬 = 0.0161
σ𝘳 = 0.0339 
α = 0.115000000000000
ϕ0 = 0.0600
ϕ1 = 16.666666666666668
μ̄ₖ = -0.034977443912449
ψ0 = 0.112733407891680
ψ1 = 0.142857142857143

# parameters for damage function settings
power = 2 
γ1 = 0.00017675
γ2 = 2. * 0.0022
γ2_plus = 2. * 0.0197
γ̄2_plus = weight * 0 + (1 - weight) * γ2_plus

σ1 = 0
σ2 = 0
ρ12 = 0
F̄ = 2
crit = 2
F0 = 1

## dont understand this part  maybe it's only for growth
# σ = np.matrix([[σ1 ** 2, ρ12], [ρ12, σ2 ** 2]])
# Σ = np.matrix([[σᵦ, 0, 0], 
#                    [0, σ1 ** 2, ρ12], 
#                    [0, ρ12, σ2 ** 2]])
# dee = np.matrix([γ1 + γ2 * F0 + γ2_plus * (F0 - F̄) ** 2 * (F0 >= 2), β𝘧, β𝘧 * F0])
# σ𝘥 = float(np.sqrt(dee * Σ * dee.T))
xi_d = -1 * (1 - κ)

# False Trasient Time step
ε = 0.1
# Specifying Tolerance level
tol = 1e-16
# Cobweb learning rate
η = 0.05

# Grids Specification

# Coarse Grids
# nR = 30
# nF = 40
# nK = 25
# R = np.linspace(R_min, R_max, nR)
# F = np.linspace(F_min, F_max, nF)
# K = np.linspace(K_min, K_max, nK)

# hR = R[1] - R[0]
# hF = F[1] - F[0]
# hK = K[1] - K[0]

# Dense Grids

R_min = 0
R_max = 9
F_min = 0
F_max = 4000
K_min = 0
K_max = 18

hR = 0.05
hF = 25
hK = 0.15

R = np.arange(R_min, R_max + hR, hR)
nR = len(R)
F = np.arange(F_min, F_max + hF, hF)
nF = len(F)
K = np.arange(K_min, K_max + hK, hK)
nK = len(K)

(R_mat, F_mat, K_mat) = np.meshgrid(R,F,K, indexing = 'ij')
stateSpace = np.hstack([R_mat.reshape(-1,1,order = 'F'),F_mat.reshape(-1,1,order = 'F'),K_mat.reshape(-1,1,order = 'F')])

# Inputs for function quad_int; Integrating across parameter distribution
quadrature = 'legendre'
n = 30
a = β𝘧 - 5 * np.sqrt(σᵦ)
b = β𝘧 + 5 * np.sqrt(σᵦ)

v0 = κ * R_mat + (1-κ) * K_mat - β𝘧 * F_mat

episode = 1
FC_Err = 1
v0 = v0_guess
q = q_guess
e_star = e_guess

while episode <= 1:# or FC_Err > tol:
    vold = v0.copy()
    # Applying finite difference scheme to the value function
    v0_dr = finiteDiff(v0,0,1,hR) 
    v0_df = finiteDiff(v0,1,1,hF)
    v0_dk = finiteDiff(v0,2,1,hK)

    v0_drr = finiteDiff(v0,0,2,hR)
    v0_drr[v0_dr < 1e-16] = 0
    v0_dr[v0_dr < 1e-16] = 1e-16
    v0_dff = finiteDiff(v0,1,2,hF)
    v0_dkk = finiteDiff(v0,2,2,hK)

    if episode == 0:
        # First time into the loop
        B1 = v0_dr - xi_d * (γ1 + γ2 * F_mat * β𝘧 + γ2_plus * (F_mat * β𝘧 - F̄) ** (power - 1) * (F_mat >= (crit / β𝘧))) * β𝘧 * np.exp(R_mat) - v0_df * np.exp(R_mat)
        C1 = - δ * κ
        e = -C1 / B1
        e_hat = e
#         Acoeff = np.exp(R_mat - K_mat)
#         Bcoeff = δ * (1-κ) / (np.exp(-R_mat + K_mat) * v0_dr * ψ0 * 0.5) + v0_dk * ϕ0 / (np.exp(-R_mat + K_mat) * v0_dr * ψ0 * 0.5)
        Acoeff = np.ones(R_mat.shape)
        Bcoeff = ((δ * (1 - κ) * ϕ1 + ϕ0 * ϕ1 * v0_dk) * δ * (1 - κ) / (v0_dr * ψ0 * 0.5) * np.exp(0.5 * (R_mat - K_mat))) / (δ * (1 - κ) * ϕ1)
        Ccoeff = -α  - 1 / ϕ1
        j = ((-Bcoeff + np.sqrt(Bcoeff ** 2 - 4 * Acoeff * Ccoeff)) / (2 * Acoeff)) ** 2
#         i = (v0_dk * ϕ0 / (np.exp(-R_mat + K_mat) * v0_dr * ψ0 * 0.5)) * (j ** 0.5) - 1 / ϕ1
        i = α - j - (δ * (1 - κ)) / (v0_dr * ψ0 * 0.5) * j ** 0.5 * np.exp(0.5 * (R_mat - K_mat))
        q = δ * (1 - κ) / (α - i - j)
    else:
        e_hat = e_star
#         j = ((α + 1 / ϕ1) * np.exp(-R_mat + K_mat) * (v0_dr * ψ0 * ψ1) / ((v0_dr * ψ0 * ψ1) * j ** (ψ1) + (δ * (1-κ) + v0_dk * ϕ0))) ** (1 / (1 - ψ1))
#         j = j * (v0_dr > 1e-8)
#         i = ((v0_dk * ϕ0 / (np.exp(-R_mat + K_mat) * v0_dr * ψ0 * ψ1)) * (j ** (1 - ψ1)) - 1 / ϕ1) * (v0_dr > 1e-8) + (v0_dr <= 1e-8) * (v0_dk * ϕ0 * α - δ * (1-κ) / ϕ1) / (δ * (1-κ) + v0_dk * ϕ0)
        
        # Cobeweb scheme to update i and j; q is 
        Converged = 0
        nums = 0
        while Converged == 0:
            i_star = (ϕ0 * ϕ1 * v0_dk / q - 1) / ϕ1
            j_star = (q * np.exp(ψ1 * (R_mat - K_mat)) / (v0_dr * ψ0 * ψ1)) ** (1 / (ψ1 - 1))
            if α > np.max(i_star + j_star):
                q_star = η * δ * (1 - κ) / (α - i_star - j_star) + (1 - η) * q
            else:
                q_star = 2 * q
            if np.max(abs(q - q_star) / η) <= 1e-5:
                Converged = 1
                q = q_star
                i = i_star
                j = j_star
            else:
                q = q_star
                i = i_star
                j = j_star
            
            nums += 1
        print('Cobweb Passed, iterations: {}, i error: {:10f}, j error: {:10f}'.format(nums, np.max(i - i_star), np.max(j - j_star)))

    a1 = np.zeros(R_mat.shape)
    b1 = xi_d * e_hat * np.exp(R_mat) * γ1
    c1 = 2 * xi_d * e_hat * np.exp(R_mat) * F_mat * γ2 
    λ̃1 = λ + c1 / ξₚ
    β̃1 = β𝘧 - c1 * β𝘧 / (ξₚ * λ̃1) -  b1 /  (ξₚ * λ̃1)
    I1 = a1 - 0.5 * np.log(λ) * ξₚ + 0.5 * np.log(λ̃1) * ξₚ + 0.5 * λ * β𝘧 ** 2 * ξₚ - 0.5 * λ̃1 * (β̃1) ** 2 * ξₚ
    #     R1 = \xi\_p.*(I1-(a1+b1.*β̃1+c1./2.*(β̃1).^2+c1./2./\lambda\tilde_1));
    R1 = 1 / ξₚ * (I1 - (a1 + b1 * β̃1 + c1 / 2 * β̃1 ** 2 + c1 / 2 / λ̃1))
    J1_without_e = xi_d * (γ1 * β̃1 + γ2 * F_mat * (β̃1 ** 2 + 1 / λ̃1)) * np.exp(R_mat)

    π̃1 = weight * np.exp(-1 / ξₚ * I1)

    def scale_2_fnc(x):
        return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e_hat)  * norm.pdf(x,β𝘧,np.sqrt(σᵦ))

    scale_2 = quad_int(scale_2_fnc, a, b, n, 'legendre')

    def q2_tilde_fnc(x):
        return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e_hat) / scale_2

    I2 = -1 * ξₚ * np.log(scale_2)

    def J2_without_e_fnc(x):
        return xi_d * np.exp(R_mat) * q2_tilde_fnc(x) * (γ1 * x + γ2 * F_mat * x ** 2 + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * norm.pdf(x,β𝘧,np.sqrt(σᵦ))

    J2_without_e = quad_int(J2_without_e_fnc, a, b, n, 'legendre')
    J2_with_e = J2_without_e * e_hat

    R2 = (I2 - J2_with_e) / ξₚ
    π̃2 = (1 - weight) * np.exp(-1 / ξₚ * I2)
    π̃1_norm = π̃1 / (π̃1 + π̃2)
    π̃2_norm = 1 - π̃1_norm

    expec_e_sum = (π̃1_norm * J1_without_e + π̃2_norm * J2_without_e)

    B1 = v0_dr - v0_df * np.exp(R_mat) - expec_e_sum
    C1 = -δ * κ
    e = -C1 / B1
    e_star = e

    J1 = J1_without_e * e_star
    J2 = J2_without_e * e_star

    I_term = -1 * ξₚ * np.log(π̃1 + π̃2)

    R1 = (I1 - J1) / ξₚ
    R2 = (I2 - J2) / ξₚ
    drift_distort = (π̃1_norm * J1 + π̃2_norm * J2)

    if weight == 0 or weight == 1:
        RE = π̃1_norm * R1 + π̃2_norm * R2
    else:
        RE = π̃1_norm * R1 + π̃2_norm * R2 + π̃1_norm * np.log(
            π̃1_norm / weight) + π̃2_norm * np.log(π̃2_norm / (1 - weight))

    RE_total = ξₚ * RE

    A = -δ * np.ones(R_mat.shape)
    ####
    B_r = -e_star + ψ0 * (j ** ψ1) * np.exp(ψ1 * (K_mat - R_mat)) - 0.5 * (σ𝘳 ** 2)
    B_f = e_star * np.exp(R_mat)
    B_k = μ̄ₖ + ϕ0 * np.log(1 + i * ϕ1) - 0.5 * (σ𝘬 ** 2)
    C_rr = 0.5 * σ𝘳 ** 2 * np.ones(R_mat.shape)
    C_ff = np.zeros(R_mat.shape)
    C_kk = 0.5 * σ𝘬 ** 2 * np.ones(R_mat.shape)
    D = δ * κ * np.log(e_star) + δ * κ * R_mat + δ * (1 - κ) * (np.log(α - i - j) + K_mat) + drift_distort + RE_total # + I_term 

    out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, v0, ε, 'False Transient')

    out_comp = out[2].reshape(v0.shape,order = "F")

    PDE_rhs = A * v0 + B_r * v0_dr + B_f * v0_df + B_k * v0_dk + C_rr * v0_drr + C_kk * v0_dkk + C_ff * v0_dff + D
    PDE_Err = np.max(abs(PDE_rhs))
    FC_Err = np.max(abs((out_comp - v0)))
#     if episode % 100 == 0:
    print("Episode {:d}: PDE Error: {:.10f}; False Transient Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}" .format(episode, PDE_Err, FC_Err, out[0], out[1]))
    episode += 1
    v0 = out_comp

print("Episode {:d}: PDE Error: {:.10f}; False Transient Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}" .format(episode, PDE_Err, FC_Err, out[0], out[1]))
print("--- %s seconds ---" % (time.time() - start_time))


### Simulation

In [ ]:
class GridInterp():

    def __init__(self, grids, values, method = 'Linear'):

        # unpacking
        self.grids = grids
        (self.xs, self.ys, self.zs) = grids
        self.nx = len(self.xs)
        self.ny = len(self.ys)
        self.nz = len(self.zs)
        
        self.values = values

        assert (self.nx, self.ny, self.nz) == values.shape, "ValueError: Dimensions not match"
        self.method = method

    def get_value(self, x, y, z):

        if self.method == 'Linear':
            
            func = RegularGridInterpolator(self.grids, self.values)
            return func([x,y,z])[0]

        elif self.method == 'Spline':

            func1 = CubicSpline(self.xs, self.values)
            yzSpace = func1(x)
            
            func2 = CubicSpline(self.ys, yzSpace)
            zSpace = func2(y)
            
            func3 = CubicSpline(self.zs, zSpace)
            return func3(z)

        else:
            raise ValueError('Method Not Supported')

In [ ]:
method = 'Linear'
T = 100
pers = 4 * T
dt = T / pers
nDims = 5
its = 1

gridpoints = (R, F, K)

e_func_r = GridInterp(gridpoints, e, method)
def e_func(x):
    return e_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

j_func_r = GridInterp(gridpoints, j, method)
def j_func(x):
    return max(j_func_r.get_value(np.log(x[0]), x[2], np.log(x[1])), 0)

i_func_r = GridInterp(gridpoints, i, method)
def i_func(x):
    return i_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

v_drfunc_r = GridInterp(gridpoints, v0_dr, method)
def v_drfunc(x):
    return v_drfunc_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

v_dtfunc_r = GridInterp(gridpoints, v0_df, method)
def v_dtfunc(x):
    return v_dtfunc_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

v_dkfunc_r = GridInterp(gridpoints, v0_dk, method)
def v_dkfunc(x):
    return v_dkfunc_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

v_func_r = GridInterp(gridpoints, v0, method)
def v_func(x):
    return v_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

pi_tilde_1_func_r = GridInterp(gridpoints, π̃1 / (π̃1 + π̃2), method)
def pi_tilde_1_func(x):
    return pi_tilde_1_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

pi_tilde_2_func_r = GridInterp(gridpoints, π̃2 / (π̃1 + π̃2), method)
def pi_tilde_2_func(x):
    return pi_tilde_2_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

def scale_2_fnc(x):
    return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e_hat)  * norm.pdf(x,β𝘧,np.sqrt(σᵦ))

scale_2 = quad_int(scale_2_fnc, a, b, n, 'legendre')

def q2_tilde_fnc(x):
    return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e_hat) / scale_2
### keyi
def base_model_drift_func(x):
    return np.exp(R_mat) * e * (γ1 * x + γ2 * x ** 2 * F_mat + γ̄2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * norm.pdf(x,β𝘧,np.sqrt(σᵦ))
base_model_drift =  quad_int(base_model_drift_func, a, b, n, 'legendre')

mean_nordhaus = β̃1
lambda_tilde_nordhaus = λ̃1
nordhaus_model_drift = (γ1 * mean_nordhaus + γ2 * (1 / lambda_tilde_nordhaus + mean_nordhaus ** 2) * F_mat) * np.exp(R_mat) * e
### keyi
def weitzman_model_drift_func(x):
    return np.exp(R_mat) * e * q2_tilde_fnc(x) * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄ ) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * norm.pdf(x,β𝘧,np.sqrt(σᵦ))
weitzman_model_drift = quad_int(weitzman_model_drift_func, a, b, n, 'legendre')

nordhaus_drift_func_r = GridInterp(gridpoints, nordhaus_model_drift, method)
def nordhaus_drift_func(x):
    return nordhaus_drift_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

weitzman_drift_func_r = GridInterp(gridpoints, weitzman_model_drift, method)
def weitzman_drift_func(x):
    return weitzman_drift_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

base_drift_func_r = GridInterp(gridpoints, base_model_drift, method)
def base_drift_func (x): 
    return base_drift_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

# function handles
####
def muR(x):
    return -e_func(x) + ψ0 * (j_func(x) * x[1] / x[0]) ** ψ1
def muK(x): 
    return (μ̄k + ϕ0 * np.log(1 + i_func(x) * ϕ1))
def muF(x):
    return e_func(x) * x[0]
def muD_base(x):
    return base_drift_func(x)
def muD_tilted(x):
    return pi_tilde_1_func(x) * nordhaus_drift_func(x) + (1 - pi_tilde_1_func(x)) * weitzman_drift_func(x)

def sigmaR(x):
    return np.zeros(x[:5].shape)
def sigmaK(x):
    return np.zeros(x[:5].shape)
def sigmaF(x):
    return np.zeros(x[:5].shape)
def sigmaD(x):
    return np.zeros(x[:5].shape)

# initial points
R_0 = 650
K_0 = 80 / α
F_0 = 870 - 580
initial_val = np.array([R_0, K_0, F_0])
D_0_base = muD_base(initial_val)
D_0_tilted = muD_tilted(initial_val)

# Set bounds
R_max_sim = np.exp(max(R))
K_max_sim = np.exp(max(K))
F_max_sim = max(F)
D_max_sim = 5.0

R_min_sim = np.exp(min(R))
K_min_sim = np.exp(min(K))
F_min_sim = min(F)
D_min_sim = -5

upperbounds = np.array([R_max_sim, K_max_sim, F_max_sim, D_max_sim, D_max_sim])
lowerbounds = np.array([R_min_sim, K_min_sim, F_min_sim, D_min_sim, D_min_sim])

hists = np.zeros([pers, nDims, its])
# hists = hists.copy()
e_hists = np.zeros([pers,its])
# e_hists = e_hists.copy()
j_hists = np.zeros([pers,its])
# j_hists = j_hists.copy()
i_hists = np.zeros([pers,its])
# i_hists = i_hists.copy()

v_dr_hists = np.zeros([pers,its])
v_dt_hists = np.zeros([pers,its])
v_dk_hists = np.zeros([pers,its])
v_hists = np.zeros([pers,its])

for iters in range(0,its):
    hist = np.zeros([pers,nDims])
    e_hist = np.zeros([pers,1])
    i_hist = np.zeros([pers,1])
    j_hist = np.zeros([pers,1])

    v_dr_hist = np.zeros([pers,1])
    v_dt_hist = np.zeros([pers,1])
    v_dk_hist = np.zeros([pers,1])
    v_hist = np.zeros([pers,1])

    hist[0,:] = [R_0, K_0, F_0, D_0_base, D_0_tilted]
    e_hist[0] = e_func(hist[0,:]) * hist[0,0]
    i_hist[0] = i_func(hist[0,:]) * hist[0,1]
    j_hist[0] = j_func(hist[0,:]) * hist[0,0]
    v_dr_hist[0] = v_drfunc(hist[0,:])
    v_dt_hist[0] = v_dtfunc(hist[0,:])
    v_dk_hist[0] = v_dkfunc(hist[0,:])
    v_hist[0] = v_func(hist[0,:])

    for tm in range(1,pers):
        shock = norm.rvs(0,np.sqrt(dt),nDims)
        # print(muR(hist[tm-1,:]))
        hist[tm,0] = cap(hist[tm-1,0] * np.exp((muR(hist[tm-1,:])- 0.5 * sum((sigmaR(hist[tm-1,:])) ** 2))* dt + sigmaR(hist[tm-1,:]).dot(shock)),lowerbounds[0], upperbounds[0])
        hist[tm,1] = cap(hist[tm-1,1] * np.exp((muK(hist[tm-1,:])- 0.5 * sum((sigmaK(hist[tm-1,:])) ** 2))* dt + sigmaK(hist[tm-1,:]).dot(shock)),lowerbounds[1], upperbounds[1])
        hist[tm,2] = cap(hist[tm-1,2] + muF(hist[tm-1,:]) * dt + sigmaF(hist[tm-1,:]).dot(shock), lowerbounds[2], upperbounds[2])
        hist[tm,3] = cap(hist[tm-1,3] + muD_base(hist[tm-1,:]) * dt + sigmaD(hist[tm-1,:]).dot(shock), lowerbounds[3], upperbounds[3])
        hist[tm,4] = cap(hist[tm-1,4] + muD_tilted(hist[tm-1,:]) * dt + sigmaD(hist[tm-1,:]).dot(shock), lowerbounds[4], upperbounds[4])

        e_hist[tm] = e_func(hist[tm-1,:]) * hist[tm-1,0]
        i_hist[tm] = i_func(hist[tm-1,:]) * hist[tm-1,1]
        j_hist[tm] = j_func(hist[tm-1,:]) * hist[tm-1,0]

        v_dr_hist[tm] = v_drfunc(hist[tm-1,:])
        v_dt_hist[tm] = v_dtfunc(hist[tm-1,:])
        v_dk_hist[tm] = v_dkfunc(hist[tm-1,:])
        v_hist[tm] = v_func(hist[tm-1,:])

    hists[:,:,iters] = hist
    e_hists[:,[iters]] = e_hist
    i_hists[:,[iters]] = i_hist
    j_hists[:,[iters]] = j_hist

    v_dr_hists[:,[iters]] = v_dr_hist
    v_dt_hists[:,[iters]] = v_dt_hist
    v_dk_hists[:,[iters]] = v_dk_hist
    v_hists[:,[iters]] = v_hist

### SCC Calculation Feyman Kac

In [ ]:
base_ = loadmat(r'C:\Users\jiamingwang\Dropbox\share with John\Final Code\Preference\SCC_solution\SCC_base_averse_weighted.mat')
base_guess = base_['v0']
worst_ = loadmat(r'C:\Users\jiamingwang\Dropbox\share with John\Final Code\Preference\SCC_solution\SCC_worst_averse_weighted.mat')
worst_guess = worst_['v0']

In [ ]:
if ξₚ > 100:  # We consider for ξₚ level over 100 as 

    MC = δ * (1-κ) / (α * np.exp(K_mat) - i * np.exp(K_mat) - j * np.exp(R_mat))
    ME = δ * κ / (e * np.exp(R_mat))
    SCC = 1000 * ME / MC
    SCC_func_r = GridInterp(gridpoints, SCC, method)
    
    def SCC_func(x): 
        return SCC_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))
    
    SCC_values = np.zeros([pers,its])
    for tm in range(pers):
        for path in range(its):   # path is its?
            SCC_values[tm, path] = SCC_func(hists[tm,:,path])

    SCC_total = np.mean(SCC_values,axis = 1)

    SCCs['SCC'] = SCC_total
    
else:

    # Base model
    def base_model_flow_func(x):
        return (γ2 * x ** 2 + γ̄2_plus * x ** 2 * ((x * F_mat - F̄) >=0)) * np.exp(R_mat) * e *  norm.pdf(x,β𝘧,np.sqrt(σᵦ))
    base_model_flow = quad_int(base_model_flow_func, a, b, n, 'legendre')
    flow_base = base_model_flow

    # input for solver

    A = -δ * np.ones(R_mat.shape)
    ####
    B_r = -e + ψ0 * (j ** ψ1) * np.exp(ψ1 * (K_mat - R_mat)) - 0.5 * (σ𝘳 ** 2)
    B_k = μ̄ₖ + ϕ0 * np.log(1 + i * ϕ1) - 0.5 * (σ𝘬 ** 2)
    B_f = e * np.exp(R_mat)
    C_rr = 0.5 * σ𝘳 ** 2 * np.ones(R_mat.shape)
    C_kk = 0.5 * σ𝘬 ** 2 * np.ones(R_mat.shape)
    C_ff = np.zeros(R_mat.shape)
    D = flow_base

    out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, base_guess, solverType='Feyman Kac')
    v0_base = out[2].reshape(v0.shape, order="F")
    v0_base = v0_base

    v0_dr_base = finiteDiff(v0_base,0,1,hR) 
    v0_df_base = finiteDiff(v0_base,1,1,hF)
    v0_dk_base = finiteDiff(v0_base,2,1,hK)

    v0_drr_base = finiteDiff(v0_base,0,2,hR)
    v0_dff_base = finiteDiff(v0_base,1,2,hF)
    v0_dkk_base = finiteDiff(v0_base,2,2,hK)

    v0_drr_base[v0_dr_base < 1e-16] = 0
    v0_dr_base[v0_dr_base < 1e-16] = 1e-16

    PDE_rhs = A * v0_base + B_r * v0_dr_base + B_f * v0_df_base + B_k * v0_dk_base + C_rr * v0_drr_base + C_kk * v0_dkk_base + C_ff * v0_dff_base + D
    PDE_Err = np.max(abs(PDE_rhs))
    print("Feyman Kac Base Model Solved. PDE Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}".format(PDE_Err, out[0], out[1]))

    # Worst Model
    mean_nordhaus = β̃1
    lambda_tilde_nordhaus = λ̃1
    def scale_2_fnc(x):
        return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e)  * norm.pdf(x,β𝘧,np.sqrt(σᵦ))

    scale_2 = quad_int(scale_2_fnc, a, b, n, 'legendre')

    def q2_tilde_fnc(x):
        return np.exp(-1 / ξₚ * xi_d * (γ1 * x + γ2 * x ** 2 * F_mat + γ2_plus * x * (x * F_mat - F̄) ** (power - 1) * ((x * F_mat - F̄) >= 0)) * np.exp(R_mat) * e) / scale_2

    nordhaus_model_flow = (γ2 * (1 / lambda_tilde_nordhaus + mean_nordhaus ** 2)) * np.exp(R_mat) * e 
    # weitzman_model_flow_func = @(x) q2_tilde_1_fnc(x) .*(gamma_2.*x.^2 +gamma_2_plus.*x.^2.*((x.*t_mat-f_bar)>=0)).*exp(r_mat).*e .*normpdf(x,beta_f,sqrt(var_beta_f));
    def weitzman_model_flow_func(x): 
        return q2_tilde_fnc(x) * (γ2 * x ** 2 + γ2_plus * x ** 2 * ((x * F_mat - F̄) >= 0 )) * np.exp(R_mat) * e * norm.pdf(x,β𝘧,np.sqrt(σᵦ))
    weitzman_model_flow = quad_int(weitzman_model_flow_func, a, b, n, 'legendre')

    I1 = a1 - 0.5 * np.log(λ) * ξₚ + 0.5 * np.log(λ̃1) * ξₚ + 0.5 * λ * β𝘧 ** 2 * ξₚ - 0.5 * λ̃1 * (β̃1) ** 2 * ξₚ
    I2 = -1 * ξₚ * np.log(scale_2)
    π̃1 = (weight) * np.exp(-1 / ξₚ * I1)
    π̃2 = (1 - weight) * np.exp(-1 / ξₚ * I2)
    π̃1_norm = π̃1 / (π̃1 + π̃2)
    π̃2_norm = 1 - π̃1_norm

    flow_tilted = π̃1_norm * nordhaus_model_flow + π̃2_norm * weitzman_model_flow

    A = -δ * np.ones(R_mat.shape)
    ####
    B_r = -e + ψ0 * (j ** ψ1) * np.exp(ψ1 * (K_mat - R_mat)) - 0.5 * (σ𝘳 ** 2)
    B_k = μ̄ₖ + ϕ0 * np.log(1 + i * ϕ1) - 0.5 * (σ𝘬 ** 2)
    B_f = e * np.exp(R_mat)
    C_rr = 0.5 * σ𝘳 ** 2 * np.ones(R_mat.shape)
    C_kk = 0.5 * σ𝘬 ** 2 * np.ones(R_mat.shape)
    C_ff = np.zeros(R_mat.shape)
    D = flow_tilted

    out = PDESolver(stateSpace, A, B_r, B_f, B_k, C_rr, C_ff, C_kk, D, worst_guess, solverType='Feyman Kac')
    v0_worst = out[2].reshape(v0.shape, order="F")
    v0_worst = v0_worst

    v0_dr_worst = finiteDiff(v0_worst,0,1,hR) 
    v0_df_worst = finiteDiff(v0_worst,1,1,hF)
    v0_dk_worst = finiteDiff(v0_worst,2,1,hK)

    v0_drr_worst = finiteDiff(v0_worst,0,2,hR)
    v0_dff_worst = finiteDiff(v0_worst,1,2,hF)
    v0_dkk_worst = finiteDiff(v0_worst,2,2,hK)
    v0_drr_worst[v0_dr_worst < 1e-16] = 0
    v0_dr_worst[v0_dr_worst < 1e-16] = 1e-16

    PDE_rhs = A * v0_worst + B_r * v0_dr_worst + B_f * v0_df_worst + B_k * v0_dk_worst + C_rr * v0_drr_worst + C_kk * v0_dkk_worst + C_ff * v0_dff_worst + D
    PDE_Err = np.max(abs(PDE_rhs))
    print("Feyman Kac Worst Model Solved. PDE Error: {:.10f}; Iterations: {:d}; CG Error: {:.10f}".format(PDE_Err, out[0], out[1]))


    # SCC decomposition

    v0_dr = finiteDiff(v0,0,1,hR) 
    v0_df = finiteDiff(v0,1,1,hF)
    v0_dk = finiteDiff(v0,2,1,hK)

    v0_drr = finiteDiff(v0,0,2,hR)
    v0_dff = finiteDiff(v0,1,2,hF)
    v0_dkk = finiteDiff(v0,2,2,hK)

    v0_drr[v0_dr < 1e-16] = 0
    v0_dr[v0_dr < 1e-16] = 1e-16

    gridpoints = (R, F, K)  # can modify

    MC = δ * (1-κ) / (α * np.exp(K_mat) - i * np.exp(K_mat) - j * np.exp(R_mat))
    ME = δ * κ / (e * np.exp(R_mat))
    SCC = 1000 * ME / MC
    SCC_func_r = GridInterp(gridpoints, SCC, method)

    def SCC_func(x): 
        return SCC_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

    ME1 = v0_dr * np.exp(-R_mat)
    SCC1 = 1000 * ME1 / MC
    SCC1_func_r = GridInterp(gridpoints, SCC1, method)
    def SCC1_func(x):
        return SCC1_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

    ME2_base = (1-κ) * v0_base
    SCC2_base = 1000 * ME2_base / MC
    SCC2_base_func_r = GridInterp(gridpoints, SCC2_base, method)
    def SCC2_base_func(x):
        return SCC2_base_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

    def V_d_baseline_func(x):
        return xi_d * (γ1 * x + γ2 * F_mat * x** 2 +
                        γ̄2_plus * x * (x * F_mat - F̄) * (power - 1)
                        * ((x * F_mat - F̄) >= 0 )) * norm.pdf(x, β𝘧, np.sqrt(σᵦ))
    V_d_baseline = quad_int(V_d_baseline_func, a, b, n, 'legendre')
    ME2b = -V_d_baseline
    SCC2_V_d_baseline = 1000 * ME2b / MC
    SCC2_V_d_baseline_func_r = GridInterp(gridpoints, SCC2_V_d_baseline, method)
    def SCC2_V_d_baseline_func(x):
        return SCC2_V_d_baseline_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

    ME2_tilt = (1-κ) * v0_worst
    SCC2_tilt = 1000 * ME2_tilt / MC
    SCC2_tilt_func_r = GridInterp(gridpoints, SCC2_tilt, method)
    def SCC2_tilt_func(x):
        return SCC2_tilt_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))


    ME2b = -expec_e_sum * np.exp(-R_mat)
    SCC2_V_d_tilt_ = 1000 * ME2b / MC
    SCC2_V_d_tilt_func_r = GridInterp(gridpoints, SCC2_V_d_tilt_, method)
    def SCC2_V_d_tilt_func(x):
        return SCC2_V_d_tilt_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))


    SCC_values = np.zeros([pers,its])
    SCC1_values = np.zeros([pers,its])
    SCC2_base_values = np.zeros([pers,its])
    SCC2_tilt_values = np.zeros([pers,its])
    SCC2_V_d_baseline_values = np.zeros([pers,its])
    SCC2_V_d_tilt_values = np.zeros([pers,its])

    for tm in range(pers):
        for path in range(its):   # path is its?
            SCC_values[tm, path] = SCC_func(hists[tm,:,path])
            SCC1_values[tm, path] = SCC1_func(hists[tm,:,path])
            SCC2_base_values[tm, path] = SCC2_base_func(hists[tm,:,path]) 
            SCC2_tilt_values[tm, path] = SCC2_tilt_func(hists[tm,:,path])
            SCC2_V_d_baseline_values[tm, path] = SCC2_V_d_baseline_func(hists[tm,:,path])
            SCC2_V_d_tilt_values[tm, path] = SCC2_V_d_tilt_func(hists[tm,:,path])

    SCC_total = np.mean(SCC_values,axis = 1)
    SCC_private = np.mean(SCC1_values,axis = 1)
    SCC2_FK_base = np.mean(SCC2_base_values,axis = 1)
    SCC2_FK_tilt = np.mean(SCC2_tilt_values,axis = 1)
    SCC2_V_d_baseline = np.mean(SCC2_V_d_baseline_values,axis = 1)
    SCC2_V_d_tilt = np.mean(SCC2_V_d_tilt_values,axis = 1)

    SCCs = {}
    SCCs['SCC'] = SCC_total
    SCCs['SCC1'] = SCC_private
    SCCs['SCC2'] = SCC2_FK_base + SCC2_V_d_baseline
    SCCs['SCC3'] = SCC2_V_d_tilt - SCC2_V_d_baseline + SCC2_FK_tilt - SCC2_FK_base

In [ ]:
SCCs['SCC3'][-1]

### Probabilities

In [ ]:
REs = {}
Dists = {}
a = β𝘧 - 5 * np.sqrt(σᵦ)
b = β𝘧 + 5 * np.sqrt(σᵦ)
a_10std = β𝘧 - 10 * np.sqrt(σᵦ)
b_10std = β𝘧 + 10 * np.sqrt(σᵦ)

RE_func_r = GridInterp(gridpoints, RE, method)
def RE_func(x):
    return RE_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

e_func_r = GridInterp(gridpoints, e, method)
def e_func(x):
    return e_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

pi_tilde_1_func_r = GridInterp(gridpoints, π̃1, method)
def pi_tilde_1_func(x):
    return pi_tilde_1_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

lambda_tilde_1_func_r = GridInterp(gridpoints, λ̃1, method)
def lambda_tilde_1_func(x):
    return lambda_tilde_1_func_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

beta_tilde_1_r = GridInterp(gridpoints, β̃1, method)
def beta_tilde_1_func(x):
    return beta_tilde_1_r.get_value(np.log(x[0]), x[2], np.log(x[1]))

RE_plot = np.zeros(pers)
weight_plot = np.zeros(pers)
beta_f_space = np.linspace(a_10std,b_10std,200)

#Relative Entropy

if damageSpec == 'low':
    nordhaus_mean = np.zeros(pers)
    nordhaus_std = np.zeros(pers)

    for tm in range(pers):
        RE_plot[tm] = RE_func(hists[tm,:,0])
        weight_plot[tm] = pi_tilde_1_func(hists[tm,:,0])
        nordhaus_mean[tm] = beta_tilde_1_func(hists[tm,:,0])
        nordhaus_std[tm] = 1 / np.sqrt(lambda_tilde_1_func(hists[tm,:,0]))

    REs['RE'] = RE_plot
    REs['Weights'] = weight_plot
    REs['Shifted Mean'] = nordhaus_mean
    REs['Shifted Std'] = nordhaus_std

else:
    for tm in range(pers):
        RE_plot[tm] = RE_func(hists[tm,:,0])
        weight_plot[tm] = pi_tilde_1_func(hists[tm,:,0])

    REs['RE'] = RE_plot
    REs['Weights'] = weight_plot


original_dist = norm.pdf(beta_f_space, β𝘧, np.sqrt(σᵦ))
Dists['Original'] = original_dist
    
# probabilities (R,K,F)

for tm in [1,100,200,300,400]:
    R0 = hists[tm-1,0,0]
    K0 = hists[tm-1,1,0]
    F0 = hists[tm-1,2,0]

    # Weitzman
    def scale_2_fnc_prob(x):
        return np.exp(-1 / ξp * xi_d * (γ1 * x + γ2 * x ** 2 *  F0 + γ2_plus * x * (x * F0 - F̄) ** (power - 1) * ((x * F0 - F̄) >= 0)) * R0 * e_func([R0, K0, F0])) * norm.pdf(x, β𝘧, np.sqrt(σᵦ))
    scale_2_prob = quad_int(scale_2_fnc_prob, a, b, n, 'legendre')

    q2_tilde_fnc_prob = np.exp(-1 / ξp * xi_d * (γ1 * beta_f_space + γ2 * beta_f_space ** 2 * F0 + γ2_plus * beta_f_space * (beta_f_space * F0 - F̄) ** (power - 1) * ((beta_f_space * F0 - F̄) >= 0)) * R0* e_func([R0, K0, F0])) / scale_2_prob * norm.pdf(beta_f_space, β𝘧, np.sqrt(σᵦ))
    weitzman = q2_tilde_fnc_prob

    # Nordhaus
    mean_distort_nordhaus = beta_tilde_1_func([R0, K0, F0]) - β𝘧
    lambda_tilde_nordhaus = lambda_tilde_1_func([R0, K0, F0])
    nordhaus = norm.pdf(beta_f_space, mean_distort_nordhaus + β𝘧, 1 / np.sqrt(lambda_tilde_nordhaus))

    # weights
    Dists_weight = pi_tilde_1_func([R0, K0, F0])
    if damageSpec == 'High':
        Dists['Weitzman_year' + str(int((tm) / 4))] = weitzman
    elif damageSpec == 'Low':
        Dists['Nordhaus_year' + str(int((tm) / 4))] = nordhaus
    elif damageSpec == 'Weighted':
        Dists['Weitzman_year' + str(int((tm) / 4))] = weitzman
        Dists['Nordhaus_year' + str(int((tm) / 4))] = nordhaus
        Dists['Weighted_year' + str(int((tm) / 4))] = nordhaus * Dists_weight + weitzman * (1 - Dists_weight)


### Save data as smart guesses

In [ ]:
# base_ = loadmat(r'C:\Users\jiamingwang\Dropbox\share with John\Final Code\Preference\SCC_solution\SCC_base_neutral_weighted.mat')
# base_guess = base_['v0']
# worst_ = loadmat(r'C:\Users\jiamingwang\Dropbox\share with John\Final Code\Preference\SCC_solution\SCC_worst_neutral_weighted.mat')
# worst_guess = worst_['v0']
check = loadmat(r'C:\Users\jiamingwang\Dropbox\share with John\Final Code\Preference\hjb_solution\Neutral_high.mat')

v0_guess = check['out_comp']
q_guess = check['q']
e_guess = check['e_hat']

In [ ]:
# smart_guess = {}
# prelin_results = {}
key = 'HighNeutral'
smart_guess[key] = {}
smart_guess[key]['v0'] = v0_guess
smart_guess[key]['q'] = q_guess
smart_guess[key]['e'] = e_guess
smart_guess[key]['base'] = None
smart_guess[key]['worst'] = None

# smart_guess['WeightedNeutral'] = {}
# smart_guess['WeightedNeutral']['v0'] = v0_guess
# smart_guess['WeightedNeutral']['q'] = q_guess
# smart_guess['WeightedNeutral']['e'] = e_guess
# smart_guess['WeightedNeutral']['base'] = base_guess
# smart_guess['WeightedNeutral']['worst'] = worst_guess

In [ ]:
smart_guess.keys()

In [ ]:
with open('./data/smartguesses.pickle', "wb") as file_:
    pickle.dump(smart_guess, file_, -1)
    
# m1 = pickle.load(open('./data/comppref.pickle', "rb", -1))

In [ ]:
pickle.load(open('./data/smartguesses.pickle', "rb", -1))['WeightedAverse'].keys()

In [ ]:
def densityPlot(key = 'Weighted'):
    years = [50, 75, 100]

    titles = ["Year {}".format(year) for year in years]

    fig = make_subplots(1, len(years), print_grid = False, subplot_titles = titles)

    dom =beta_f_space
    inds = ((dom>=0) & (dom<=5e-3))

    for i, year in enumerate(years):
        # data = loadmat("{}/50-50 weight/Dist_{}yr.mat".format(quad_rule, year))
        data = Dists
        if key == 'Weighted': 
            if i == 0:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = True, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Nordhaus_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'Low Damage Function', line = dict(color = 'red', dash='dashdot', width = 3), showlegend = True, legendgroup = 'Low Damage Function')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Weitzman_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'High Damage Function', line = dict(color = 'green', dash='dash', width = 3), showlegend = True, legendgroup = 'High Damage Function')
            else:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = False, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Nordhaus_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'Low Damage Function', line = dict(color = 'red', dash='dashdot', width = 3), showlegend = False, legendgroup = 'Low Damage Function')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Weitzman_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'High Damage Function', line = dict(color = 'green', dash='dash', width = 3), showlegend = False, legendgroup = 'High Damage Function')

        elif key == 'High':
            if i == 0:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = True, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Weitzman_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'High Damage Function', line = dict(color = 'green', dash='dash', width = 3), showlegend = True, legendgroup = 'High Damage Function')
            else:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = False, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Weitzman_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'High Damage Function', line = dict(color = 'green', dash='dash', width = 3), showlegend = False, legendgroup = 'High Damage Function')


        elif key == 'Low':
            if i == 0:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = True, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Nordhaus_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'Low Damage Function', line = dict(color = 'red', dash='dashdot', width = 3), showlegend = True, legendgroup = 'Low Damage Function')
            else:
                fig.add_scatter(x = dom[inds] * 1000, y = data['Original'][inds], row = 1, col = i + 1,
                    name = 'Original Distribution', line = dict(color = '#1f77b4', width = 3), showlegend = False, legendgroup = 'Original Distribution')
                fig.add_scatter(x = dom[inds] * 1000, y = data['Nordhaus_year' + str(year)][inds], row = 1, col = i + 1,
                    name = 'Low Damage Function', line = dict(color = 'red', dash='dashdot', width = 3), showlegend = False, legendgroup = 'Low Damage Function')

    fig['layout'].update(title = key + " Damage Specification", showlegend = True, titlefont = dict(size = 20), height = 400)

    for i in range(len(years)):

        fig['layout']['yaxis{}'.format(i+1)].update(showgrid = False)
        fig['layout']['xaxis{}'.format(i+1)].update(showgrid = False)

    fig['layout']['yaxis1'].update(title=go.layout.yaxis.Title(
                                    text="Probability Density", font=dict(size=16)))
    fig['layout']['xaxis2'].update(title=go.layout.xaxis.Title(
                                    text="Climate Sensitivity", font=dict(size=16)), showgrid = False)

    fig = go.FigureWidget(fig)
    iplot(fig)

In [ ]:
densityPlot(damageSpec)

In [ ]:
def SCCDecomposePlot(key = 'Weighted'):

    if key == 'Low':

        data = SCCs
        x1, y1, x2, y2, x3, y3 = 60, 195, 93, 330, 96, 100

    elif key == 'Weighted':

        data = SCCs
        x1, y1, x2, y2, x3, y3 = 60, 320, 80, 315, 90, 350

    elif key == 'High':

        data = SCCs
        x1, y1, x2, y2, x3, y3 = 60, 340, 93, 495, 96, 430


    total_SCC = np.array(data['SCC'])
    external_SCC = np.array(data['SCC2'])
    uncertainty_SCC = np.array(data['SCC3'])
    private_SCC = np.array(data['SCC1'])
    x = np.linspace(0,100,400)

    total = go.Scatter(x = x, y = total_SCC,
                   name = 'Total', line = dict(color = '#1f77b4', dash = 'solid', width = 3),\
                       showlegend = False)
    external = go.Scatter(x = x, y = external_SCC,
                   name = 'Ambiguity', line = dict(color = 'red', dash = 'dot', width = 3),\
                          showlegend = False)
    uncertainty = go.Scatter(x = x, y = uncertainty_SCC,
                   name = 'No Ambiguity', line = dict(color = 'green', dash = 'dashdot', width = 3),\
                             showlegend = False)
    private = go.Scatter(x = x, y = private_SCC,
                   name = 'Private', line = dict(color = 'black', width = 3),\
                         showlegend = False)

    annotations=[dict(x=x1, y=y1, text="Total", textangle=0, ax=-100,
                ay=-75, font=dict(color="black", size=12), arrowcolor="black",
                arrowsize=3, arrowwidth=1, arrowhead=1),

                dict(x=x2, y=y2, text="Ambiguity", textangle=0, ax=-100,
                ay=0, font=dict(color="black", size=12), arrowcolor="black",
                arrowsize=3, arrowwidth=1, arrowhead=1),

                dict(x=x3, y=y3, text="No Ambiguity", textangle=0, ax=-80,
                ay=80, font=dict(color="black", size=12), arrowcolor="black",
                arrowsize=3, arrowwidth=1, arrowhead=1)]

    layout = dict(title = 'Social Cost of Carbon, {} Damage Specification'.format(key),
                  titlefont = dict(size = 20),
                  xaxis = go.layout.XAxis(title=go.layout.xaxis.Title(
                                    text='Years', font=dict(size=16)),
                                         tickfont=dict(size=12), showgrid = False),
                  yaxis = go.layout.YAxis(title=go.layout.yaxis.Title(
                                    text='Dollars per Ton of Carbon', font=dict(size=16)),
                                         tickfont=dict(size=12), showgrid = False), 
                  annotations=annotations
                  )

    fig = dict(data = [total, external, uncertainty], layout = layout)
    iplot(fig)
    

In [ ]:
SCCDecomposePlot(damageSpec)

In [ ]:
def emissionPlot(damageSpec, ξ):

    colors = {'High': 'red', 'Low': 'green', 'Weighted': '#1f77b4'}
    lines = {'Averse': 'solid', "Neutral": 'dashdot'}

    # damageSpecs = ['High', 'Low', 'Weighted']
    # aversionSpecs = ['Averse', 'Neutral']
    # colors = ['green', '#1f77b4', 'red']
    # lines = ['solid', 'dashdot'] 

    x = np.linspace(0, 100, 400)
    data = []

    data.append(go.Scatter(x = x, y = e_hists[:,0], name = damageSpec +  ' Damage w/ ξ= {}'.format(ξ),
        line = dict(width = 2, dash = 'solid', color = colors[damageSpec]), showlegend = True))

    layout = dict(title = 'Emissions Plot with {} Damage Setting, ξ = {}'.format(damageSpec, ξ),
      titlefont = dict(size = 20),
      xaxis = go.layout.XAxis(title=go.layout.xaxis.Title(
                        text='Years', font=dict(size=16)),
                             tickfont=dict(size=12), showgrid = False, showline = True),
      yaxis = go.layout.YAxis(title=go.layout.yaxis.Title(
                        text='Gigatons of Carbon', font=dict(size=16)),
                             tickfont=dict(size=12), showgrid = False),
      legend = dict(orientation = 'h', y = 1.15)
      )

    fig = dict(data = data, layout = layout)
    iplot(fig)

In [ ]:
emissionPlot(damageSpec, ξp)